At first we load the data and do the pre-processing on it 
1. check if there is any NULL in the dataset (Because it is the MNIST dataset and it is popular we did not check this part)
2. normalize the dataset with z-score and transfer the dataset to the new space with the mean of zero and the standard deviation of 1
(Normalizing is not necessary in this project because all the features are in the same range from 0 to 255 but we do not do it here because it cause a problem in the fiting of the autoencoder and it needs integers)

In [55]:
import pandas as pd
from scipy.stats import zscore

# Load the Data Set
data_labeled_train = pd.read_csv('labeled_train_set.csv')
data_labeled_test = pd.read_csv('test_set.csv')
data_unlabeled_train = pd.read_csv('unlabeled_train_set.csv')

# Conver the Data Set to Numpy Array and Spliting Labels and Features
X_train_labeled = data_labeled_train.iloc[: , 1:].values
y_train_labeled = data_labeled_train.iloc[: , 0].values
X_test_labeled = data_labeled_test.iloc[: , 1:].values
y_test_labeled = data_labeled_test.iloc[: , 0].values
X_train_unlabeled = data_unlabeled_train.iloc[:, :].values


Now we build an Auto Encoder and use it for Feature Selection and by that Reduce the dimentionality of the data
We are going to implement it in 5 layers: 
1. Input Layer
2. 3 Hidden Layers (128 , 16, 128)
3. Output Layer
So now it is obvious how the architecture of our Encoder and also Decoder is

In [57]:
from tensorflow import keras
from tensorflow.keras import layers

# Implimenting the Auto Encoder with the architecture we talked about

# Defining the Input Image dimention
input_image = keras.Input(shape=(784,))

# Define the Encoder
encoded = layers.Dense(128, activation='relu')(input_image)
encoded = layers.Dense(16, activation='relu')(encoded)

# Build the Encoder
encoder = keras.Model(input_image, encoded)

# Define the Decoder 
encoded_input = keras.Input(shape=(16,))
decoded = layers.Dense(128, activation='relu')(encoded_input)
decoded = layers.Dense(784, activation='sigmoid')(decoded)

# Build the Decoder 
decoder = keras.Model(encoded_input, decoded)

# Build the Auto Encoder
auto_encoder = keras.Model(input_image, decoder(encoder(input_image)))

# Compile the Auto Encoder
auto_encoder.compile(optimizer="adam", loss="binary_crossentropy")

# Fit the Data on our Auto Encoder
log = auto_encoder.fit(X_train_labeled, X_train_labeled, epochs=50, batch_size=256,
shuffle=True, validation_data=(X_test_labeled, X_test_labeled))
    

Epoch 1/50
71/71 [==============================] - 3s 30ms/step - loss: -1250878.6250 - val_loss: -8148861.5000
Epoch 2/50
71/71 [==============================] - 2s 25ms/step - loss: -84777888.0000 - val_loss: -282595808.0000
Epoch 3/50
71/71 [==============================] - 1s 18ms/step - loss: -944950528.0000 - val_loss: -2164544000.0000
Epoch 4/50
71/71 [==============================] - 2s 23ms/step - loss: -4638723072.0000 - val_loss: -8611849216.0000
Epoch 5/50
71/71 [==============================] - 2s 26ms/step - loss: -14802934784.0000 - val_loss: -24157425664.0000
Epoch 6/50
71/71 [==============================] - 1s 20ms/step - loss: -36543922176.0000 - val_loss: -54645645312.0000
Epoch 7/50
71/71 [==============================] - 1s 21ms/step - loss: -76108349440.0000 - val_loss: -107067572224.0000
Epoch 8/50
71/71 [==============================] - 2s 27ms/step - loss: -140604850176.0000 - val_loss: -189103734784.0000
Epoch 9/50
71/71 [=============================

After Fitting of the Auto Encoder is done, we encode the data set with it (reduce its dimentionality to 16)
P.S. I decide to reduce the dimentionality of the unlabeled data and then implement clustring algorithm on it to decrease the time of the running time

In [58]:
encoded_X_train_labeled = encoder.predict(X_train_labeled)
encoded_X_test_labeled = encoder.predict(X_test_labeled)
encoded_X_train_unlabeled = encoder.predict(X_train_unlabeled)


1313/1313 [==============================] - 4s 3ms/step
[[7.4275728e+07 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 7.3842136e+07]
 [1.8476669e+08 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.8368750e+08]
 [2.1643147e+08 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 2.1516738e+08]
 ...
 [1.4252397e+08 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.4169155e+08]
 [1.6324323e+08 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.6228952e+08]
 [8.5809712e+07 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 8.5308656e+07]]
18000
16
